In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn import preprocessing
import seaborn as sns
import matplotlib.pyplot as plt
from Utils.preprocesado1 import preprocesado as prep1

trenes = pd.read_csv("/Users/valdecantos/Documents/The_Bridge/Alumno_Folder/Alumno_Curso/datascience_thebridge_9_21/Proyecto ML/viajes-en-tren-sample.csv")


#QUITAR TILDES CIUDADES DE ORIGEN
trenes['origin'] = trenes['origin'].str.replace('CÓRDOBA','CORDOBA')
trenes['origin'] = trenes['origin'].str.replace('LEÓN','LEON')
trenes['origin'] = trenes['origin'].str.replace('MÁLAGA','MALAGA')


#QUITAR TILDES CIUDADES DE DESTINO

trenes['destination'] = trenes['destination'].str.replace('CÓRDOBA','CORDOBA')
trenes['destination'] = trenes['destination'].str.replace('LEÓN','LEON')
trenes['destination'] = trenes['destination'].str.replace('MÁLAGA','MALAGA')


#REDUCIR DE TIPOS DE TRENES
trenes['vehicle_type'] = trenes['vehicle_type'].str.replace('LD-AVE','AVE')
trenes['vehicle_type'] = trenes['vehicle_type'].str.replace('AVE-LD','AVE')
trenes['vehicle_type'] = trenes['vehicle_type'].str.replace('MD-AVE','AVE')
trenes['vehicle_type'] = trenes['vehicle_type'].str.replace('AVE-MD','AVE')
trenes['vehicle_type'] = trenes['vehicle_type'].str.replace('REGIONAL','REG.EXP')
trenes['vehicle_type'] = trenes['vehicle_type'].str.replace('MD-LD','MD')
trenes['vehicle_type'] = trenes['vehicle_type'].str.replace('LD-MD','LD')
trenes['vehicle_type'] = trenes['vehicle_type'].str.replace('LD-AVANT','AVANT')
trenes['vehicle_type'] = trenes['vehicle_type'].str.replace('REGIONAL.','REG.EXP')
trenes['vehicle_type'] = trenes['vehicle_type'].str.replace('Intercity','INTERCITY')
trenes['vehicle_type'] = trenes['vehicle_type'].str.replace('REG.EXP.','REG.EXP')

#REDUCIR TIPOS DE VAGON
trenes['vehicle_class'] = trenes['vehicle_class'].str.replace('Turista - Turista Plus','Turista')
trenes['vehicle_class'] = trenes['vehicle_class'].str.replace('Turista Plus - Turista','Turista Plus')

    
   



le = preprocessing.LabelEncoder()

#TRANSFORMAR CIUDADES DE ORIGEN
le.fit(trenes['origin'])
trenes['origin_le'] = le.transform(trenes['origin'])

#TRANSFORMAR CIUDADES DE DESTINO
le.fit(trenes['destination'])
trenes['destination_le'] = le.transform(trenes['destination'])

#TRANSFORMAR TIPOS DE TRENES
le.fit(trenes['vehicle_type'])
trenes['vehicle_type_le'] = le.transform(trenes['vehicle_type'])

#TRANSFORMAR TIPOS DE VAGON
le.fit(trenes['vehicle_class'])
trenes['vehicle_class_le'] = le.transform(trenes['vehicle_class'])

#TRANSFORMAR TARIFAS
le.fit(trenes['fare'])
trenes['fare_le'] = le.transform(trenes['fare'])





trenes['departure'] = pd.to_datetime(trenes['departure'])
trenes['Month'] = trenes['departure'].dt.month
trenes['Day'] = trenes['departure'].dt.day
trenes['Day of the week'] = trenes['departure'].dt.dayofweek
trenes['Hour'] = trenes['departure'].dt.hour


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:33: FutureWarning: The default value of regex will change from True to False in a future version.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:35: FutureWarning: The default value of regex will change from True to False in a future version.


In [2]:
df = trenes.drop(['company', 'origin', 'destination', 'departure', 'arrival', 'vehicle_type', 'vehicle_class', 'fare', 'meta', 'insert_date'], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 12 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   duration          1000000 non-null  float64
 1   price             1000000 non-null  float64
 2   seats             1000000 non-null  int64  
 3   origin_le         1000000 non-null  int64  
 4   destination_le    1000000 non-null  int64  
 5   vehicle_type_le   1000000 non-null  int64  
 6   vehicle_class_le  1000000 non-null  int64  
 7   fare_le           1000000 non-null  int64  
 8   Month             1000000 non-null  int64  
 9   Day               1000000 non-null  int64  
 10  Day of the week   1000000 non-null  int64  
 11  Hour              1000000 non-null  int64  
dtypes: float64(2), int64(10)
memory usage: 91.6 MB


In [3]:
df.head(2)

,duration,price,seats,origin_le,destination_le,vehicle_type_le,vehicle_class_le,fare_le,Month,Day,Day of the week,Hour
0,2.75,64.25,325,3,0,2,0,2,1,26,1,16
1,1.78,33.50,289,0,8,2,0,2,2,12,4,8


In [4]:
X = df[['duration','seats','origin_le','destination_le','vehicle_type_le','vehicle_class_le','fare_le','Month','Day','Day of the week','Hour']]
y = df["price"]

In [10]:
from sklearn import svm
from sklearn.svm import LinearSVR
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, accuracy_score, r2_score


X_train, X_test, y_train, y_test = train_test_split(X, y , test_size = 0.2)

svm_reg = LinearSVR(epsilon=1.5, random_state=42)

In [6]:
svm_reg.fit(X_train, y_train)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/svm/_base.py:1201: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


LinearSVR(epsilon=1.5, random_state=42)

In [7]:
y_pred = svm_reg.predict(X_test)
y_pred

array([29.25353528, 44.62517725, 42.9841901 , ..., 75.60839531,
       70.06261326, 54.58674405])

In [8]:
print("MSE", mean_squared_error(y_test, svm_reg.predict(X_test)))
print("RMSE", np.sqrt(mean_squared_error(y_test, svm_reg.predict(X_test))))
print("MAE", mean_absolute_error(y_test, svm_reg.predict(X_test)))
print('R2 score', r2_score(y_test, svm_reg.predict(X_test)))

MSE 432.95236601130233
RMSE 20.807507443499865
MAE 17.855245498148278
R2 score -0.12203768406004922


In [13]:
parameters = {
    'kernel': ['rbf', 'poly'],
    'C': [0.001, 0.1, 0.5, 1, 5, 10, 100],
    'degree': [2,3,4],
    'gamma': ['scale', 'auto'],
    'epsilon': [0.1, 0.3, 0.5]
}

svr = svm.SVR()

Grid_reg = GridSearchCV(estimator = svr,
                  param_grid = parameters,
                  n_jobs = -1,
                  cv = 5)

Grid_reg.fit(X_train, y_train)

KeyboardInterrupt: 